In [1]:
import pandas as pd

dados = pd.read_excel('bundesliga_stats.xlsx')
dados.head()

,Squad,MP,W,D,L,GF,GA,GD,Pts,Pts/MP,...,xGA,xGD,xGD/90,Attendance,data,time_casa,gols_casa,gols_fora,time_fora,resultado
0,Bayern Munich,14.0,10.0,3.0,1.0,42.0,12.0,30.0,33.0,236.0,...,91.0,221.0,158.0,75.000,15.12. 15:30,RB Leipzig,2,1,Eint Frankfurt,vitoria_casa
1,Leverkusen,14.0,8.0,5.0,1.0,32.0,20.0,12.0,29.0,207.0,...,159.0,98.0,70.0,29.830,15.12. 13:30,Dortmund,1,1,Hoffenheim,empate
2,Eint Frankfurt,14.0,8.0,3.0,3.0,34.0,20.0,14.0,27.0,193.0,...,223.0,46.0,33.0,57.683,15.12. 11:30,Heidenheim,1,3,Stuttgart,vitoria_visitante
3,RB Leipzig,14.0,8.0,3.0,3.0,23.0,15.0,8.0,27.0,193.0,...,188.0,14.0,10.0,44.258,14.12. 14:30,St. Pauli,0,2,Werder Bremen,vitoria_visitante
4,Freiburg,14.0,7.0,3.0,4.0,20.0,19.0,1.0,24.0,171.0,...,152.0,57.0,41.0,34.100,14.12. 11:30,Augsburg,0,2,Bayer Leverkusen,vitoria_visitante


In [2]:
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb

# Coluna target é o resultado
# X são as colunas independentes e Y alvo
x = dados.iloc[:,:20]
y = dados['resultado']

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=.3, random_state=100)

In [3]:
LabelEncoder = LabelEncoder()
 
#seleciono as colunas apenas pelo tipo objeto
a = dados.select_dtypes(include="object")
 
#para todo i que estiver nas colunas categoricas 
for i in a:
  if i != "resultado":
    dados[i]= LabelEncoder.fit_transform(dados[i])
 

In [4]:
categoricos = ['Squad', 'data', 'time_casa', 'time_fora']

for i in categoricos:
    if i in X_train.columns:
        all_values = pd.concat([X_train[i], X_test[i]], axis=0)
        LabelEncoder.fit(all_values)
        X_train[i] = LabelEncoder.transform(X_train[i])
        X_test[i] = LabelEncoder.transform(X_test[i])

In [5]:
y_train_encod = LabelEncoder.fit_transform(y_train)

In [6]:
print(X_train.dtypes)

Squad           int32
MP            float64
W             float64
D             float64
L             float64
GF            float64
GA            float64
GD            float64
Pts           float64
Pts/MP        float64
xG            float64
xGA           float64
xGD           float64
xGD/90        float64
Attendance    float64
data            int32
time_casa       int32
gols_casa       int64
gols_fora       int64
time_fora       int32
dtype: object


In [7]:
xgb = xgb.XGBClassifier(booster='gbtree', enable_categorical=True)
scores = 100 * cross_val_score(xgb, X_train, y_train_encod)
average_score = scores.mean()
print(f"Pontuação média de validação cruzada:{average_score:.2f}%")

Pontuação média de validação cruzada:85.14%


In [8]:
# Treinando o modelo, usa o metodo fit em X_train e y_train
xgb.fit(X_train, y_train_encod)

XGBClassifier(base_score=None, booster='gbtree', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=True, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [9]:
# Prevendo o teste(X_test) com predict

previsao = xgb.predict(X_test)

In [10]:
# Convertendo os valores numericos em categoricos usando o inverse da LabelEncoding

transform = LabelEncoder.inverse_transform(previsao)

In [11]:
# Criar o DataFrame com as classes reais e preditas
resultado = pd.DataFrame({
    'Real': y_test.values,  # Valores reais já estão com os nomes originais
    'Predito': transform  # Converter para os nomes das classes
})
resultado

,Real,Predito
0,vitoria_casa,vitoria_casa
1,vitoria_casa,vitoria_casa
2,vitoria_visitante,vitoria_visitante
3,vitoria_visitante,vitoria_visitante
4,vitoria_casa,vitoria_casa
5,empate,empate
6,vitoria_casa,vitoria_casa
7,vitoria_visitante,vitoria_visitante
8,empate,vitoria_casa
9,empate,empate
